In [67]:
import requests
import json
import time
import pandas as pd
from IPython.display import clear_output

loop_count = 250
beer_count = 0

beer_df = pd.DataFrame(columns=['id','name','abv','ph','ibu', 'first_brewed', 'hops'])

for x in range(150, loop_count+1):
    #Use IPython custome clear output to minimise outputs
    clear_output(wait=True)
    
    #Get JSON using URL Request
    url_api = 'https://api.punkapi.com/v2/beers/' + str(x)
    response = requests.get(url=url_api, headers = {'Content-Type': 'application/json', 'Accept': 'application/json'})
    
    #If error, API will return differently structured JSON rather than a HTTP error
    try:
        #Store JSON, grabbing first element (due to JSON structure from API)
        json_beer = json.loads(response.text)[0]
    except:
        #Error - so skip (but show ID with error)
        print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far! No beer with ID: ",x)
        continue
        
    #Assign beer data into Dictionary for later analysis
    beer_dict = {}
    
    #Create lists for hops and yeast
    hops_list = []
    yeast_list = []
    
    #Store beer info from JSON
    beer_dict['id'] = json_beer['id']
    beer_dict['name'] = json_beer['name']
    beer_dict['abv'] = json_beer['abv']
    beer_dict['ph'] = json_beer['ph']
    beer_dict['ibu'] = json_beer['ibu'] 
    beer_dict['first_brewed'] = json_beer['first_brewed']
    
    #Use a List comprehension to get all hops types per beer
    hops_list = [hop['name'] for hop in json_beer['ingredients']['hops']]
    
    #Remove hop duplicates (by creating a set) and put into array
    hops_list = list(set(hops_list))
    beer_dict['hops'] = hops_list
    
    #Append to dataframe
    temp_df  = pd.DataFrame([beer_dict], columns=beer_list.keys())
    beer_df = beer_df.append(temp_df)
    
    #Add beer count
    beer_count  += 1
    
    #Print progress
    print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far!")
    
    #Wait fixed time to prevent API throttling / restricting usage
    time.sleep(.75)

#Reset index of Dataframe (ensuring that old index doesn't get added as a column)
beer_df = beer_df.reset_index(drop=True)

Grabbing Punk beer data. Current progress: 250 / 250 .  101  beer(s) discovered so far!


In [99]:
import pandas as pd

#Create new dataframe to understand hops brewed by year
hops_df = pd.DataFrame(columns=['name', 'year'])

#Iterate each row in the raw beer dataframe
for index, row in beer_df.iterrows():
    
    #As multiple hops can be in one brew, second loop to iterate through list within beer dataframe row
    for hop_name in row['hops']:
        
        #Create temporary dictionary to be used to append to the hops dataframe
        temp_dict = {}

        #Get name of hop
        temp_dict['name'] = hop_name
        
        #Get year of brew, difficulty here is inconsistent formatting in data (sometimes MM/YYYY or YYYY)
        #Therefore search for "/" and split if required 
        temp_dict['year'] = beer_df.loc[index, 'first_brewed']
        if "/" in temp_dict['year']:
            temp_dict['year'] = temp_dict['year'].split("/")[1]
            
        #Append temperorary dictionary to hops dataframe
        temp_df  = pd.DataFrame([temp_dict], columns=hops_df.keys())
        hops_df = hops_df.append(temp_df)

#Reset index of Dataframe (ensuring that old index doesn't get added as a column)
hops_df = hops_df.reset_index(drop=True)  

#Change hops dataframe structure - years as columns and hop name as index
hops_df = pd.crosstab(index=hops_df['name'], columns=hops_df['year'])

#Reset index of dataframe to ensure name is a column (as well as index name)
hops_df = hops_df.reset_index()

#Print progress
print("Hops dataframe completed!")

Hops dataframe completed!


In [149]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

#Get list of years required for slider (removing first item as it is 'Name')
years = list(hops_df)[1:]

#Get list of hops names
hops = hops_df['name'].tolist()

In [150]:
#Create and upload required grid to Plotly
grid = Grid(hops_df)
url = py.grid_ops.upload(grid, 'gapminder_grid'+str(time.time()), auto_open=False)
print(url)

#Create figure dictionary
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

#Fill in layout details (via Figure dictionairy)
figure['layout']['xaxis'] = {'title': 'Hop', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'title': 'Number of Times in a Brew', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
figure['layout']['title'] = 'Brewdog Hop Choice per Year'

#Create slider dictionairy
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

#Add play and pause buttons by updating Layout
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

https://plot.ly/~cjmcguicken/10/


In [155]:
col_name_template = '{column}'
for year in years:
    frame = {'data': [], 'name': str(year)}
    x_list = []
    y_list = []
    
    for hop in hops:
        x_list.append(grid.get_column_reference(col_name_template.format(column='name')))
        y_list.append(grid.get_column_reference(col_name_template.format(column=year)))
     
    frame['data'].append(go.Bar(
            x=x_list,
            y=y_list
    ))

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [156]:
py.icreate_animations(figure, 'barchart example')

PlotlyRequestError: Figure field is invalid. Reason: Raw data arrays are not allowed at this endpoint. Use grid references instead. Raw data found at the following paths in the figure [('frames', 0, 'data', 0, 'x'), ('frames', 0, 'data', 0, 'y'), ('frames', 1, 'data', 0, 'x'), ('frames', 1, 'data', 0, 'y'), ('frames', 2, 'data', 0, 'x'), ('frames', 2, 'data', 0, 'y'), ('frames', 3, 'data', 0, 'x'), ('frames', 3, 'data', 0, 'y'), ('frames', 4, 'data', 0, 'x'), ('frames', 4, 'data', 0, 'y'), ('frames', 5, 'data', 0, 'x'), ('frames', 5, 'data', 0, 'y'), ('frames', 6, 'data', 0, 'x'), ('frames', 6, 'data', 0, 'y'), ('frames', 7, 'data', 0, 'x'), ('frames', 7, 'data', 0, 'y'), ('frames', 8, 'data', 0, 'x'), ('frames', 8, 'data', 0, 'y'), ('frames', 9, 'data', 0, 'x'), ('frames', 9, 'data', 0, 'y'), ('frames', 10, 'data', 0, 'x'), ('frames', 10, 'data', 0, 'y'), ('frames', 11, 'data', 0, 'x'), ('frames', 11, 'data', 0, 'y'), ('frames', 12, 'data', 0, 'x'), ('frames', 12, 'data', 0, 'y'), ('f